# SignXAI2 PyTorch - Time Series (ECG) Analysis

This tutorial demonstrates ECG time series analysis with dynamic method parsing.

## Features:
- 12-lead ECG visualization
- Dynamic method parameters
- Time series specific methods

In [ ]:
import torch
import numpy as np
from signxai.api import explain
from signxai.torch_signxai.utils import remove_softmax
from utils.ecg_data import load_and_preprocess_ecg
from utils.ecg_visualization import plot_ecg

In [ ]:
# Dynamic method examples for ECG
ecg_methods = [
    'gradient',
    'gradient_x_input_x_sign_mu_neg_0_5',
    'smoothgrad_noise_0_3_samples_50',
    'integrated_gradients_steps_100',
    'lrp_epsilon_0_25'
]

# Load ECG data
ecg_data = load_and_preprocess_ecg(record_id='03509_hr')

# Generate explanations
for method in ecg_methods:
    explanation = explain(model, ecg_data, method_name=method)
    plot_ecg(ecg_data, explanation, title=method)